In [1]:
import xlrd
import sklearn
from sklearn.neural_network import MLPClassifier
import numpy as np
import os
import scipy.stats
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from keras.models import load_model
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras import backend as K
from keras import regularizers
import numpy as np
from collections import Counter
from sklearn.linear_model import LogisticRegression
import scipy.stats
import random
import tensorflow as tf
from collections import defaultdict
from scipy import stats
import matplotlib.pyplot as plt 
from keras import optimizers
import time

/anaconda3/envs/nlu/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/envs/nlu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# open the excel file
loc = ("Premier_League_Data_most_recent_copy.xlsx")
wb = xlrd.open_workbook(loc)

In [3]:
num_parameters = 22
num_teams = 20
num_stats_sheets = 3
num_total_sheets = 4

In [4]:
stats_sheets = []
for i in range(0, num_stats_sheets):
    stats_sheets.append(wb.sheet_by_index(i))

    

In [5]:
stats_by_season = []
teams_per_season = []
promoted_per_season = []
relegated_per_season = []

In [6]:
for m in range(1, num_stats_sheets):
    stats_this_year = {}
    teams_this_year = []
    promoted_this_year = []
    relegated_this_year = []
    for i in range(1, num_teams + 1):
        team = stats_sheets[m].cell_value(i, 0)
#         print(team)
        teams_this_year.append(team)
        stats_this_year[team] = []
        for j in range(1, num_parameters + 1):
            stats_this_year[team].append(stats_sheets[m].cell_value(i, j))
            
    relegated_this_year.append(teams_this_year[num_teams - 3])
    relegated_this_year.append(teams_this_year[num_teams - 2])
    relegated_this_year.append(teams_this_year[num_teams - 1])

#     print(relegated_this_year)
    promoted_this_year.append(stats_sheets[m].cell_value(num_teams + 3, 0))
    promoted_this_year.append(stats_sheets[m].cell_value(num_teams + 4, 0))
    promoted_this_year.append(stats_sheets[m].cell_value(num_teams + 5, 0))

    stats_by_season.append(stats_this_year)
    teams_per_season.append(teams_this_year)
#     print(promoted_this_year)
    promoted_per_season.append(promoted_this_year)
    relegated_per_season.append(relegated_this_year)
    # print(teams_this_year)
#print(teams_per_season[0])
promoted_per_season.append(['AFC Bournemouth', 'Watford', 'Norwich City'])
print(promoted_per_season)



[['Newcastle United', 'Brighton and Hove Albion', 'Huddersfield Town'], ['Burnley', 'Middlesbrough', 'Hull City'], ['AFC Bournemouth', 'Watford', 'Norwich City']]


In [7]:
score_sheets = []
for i in range(num_stats_sheets + 4, num_total_sheets + 5):
    score_sheets.append(wb.sheet_by_index(i))

In [29]:
all_results = []
all_stats = []
all_outcomes = []
stats_pairs_by_season = []
outcomes_by_season = []

for i in range (0 , 2):
    print("STARTING OVER")
    results = {}
    wins = 0
    losses = 0
    draws = 0
    for j in range (1, num_teams * (num_teams - 1) + 1):
        # print(score_sheets[i].cell_value(j, 1))
        home_team = score_sheets[i].cell_value(j, 1).strip()
        away_team = score_sheets[i].cell_value(j, 2).strip()
        result = score_sheets[i].cell_value(j, 5)
        
        if result == "W":
            wins +=1 
            results[(home_team, away_team)] = 1
        elif result == "L":
            losses+=1
            results[(home_team, away_team)] = -1
        else:
            draws+=1
            results[(home_team, away_team)] = 0
    all_results.append(results)
#     print("results: ", results)
    stats = []
    outcomes = []
    counter = 0
    for key in results.keys():
        counter = counter + 1
        # each home-away pair for that season
        home_team = key[0].strip()
#         print("home team: ", home_team)
        away_team = key[1].strip()
#         print("away team: ", away_team)
        prev_season_stats = stats_by_season[i]
        if home_team in prev_season_stats.keys():
            home_stats = prev_season_stats[home_team]
        else:
#             print(promoted_per_season[i])
#             print(promoted_per_season[i+1])
#             print(promoted_per_season)
            promoted_index = promoted_per_season[i].index(home_team)
#             print("promoted_index: ", promoted_index)
            promoted_team = promoted_per_season[i+1][promoted_index]
#             print(prev_season_stats)
            home_stats = prev_season_stats[promoted_team]
            
#             promoted_index = promoted_per_season[i+1].index(home_team)
#            #  print(promoted_index)
#             promoted_team = promoted_per_season[i+2][promoted_index] # out of bounds error?
#             home_stats = prev_season_stats[promoted_team]


        if away_team in prev_season_stats.keys():
            away_stats = prev_season_stats[away_team]
        else:
            promoted_index = promoted_per_season[i].index(away_team)
            promoted_team = promoted_per_season[i+1][promoted_index]
            away_stats = prev_season_stats[promoted_team]
            
#             promoted_index = promoted_per_season[i + 1].index(away_team)
#             promoted_team = promoted_per_season[i + 2][promoted_index] # out of bounds error?
#             away_stats = prev_season_stats[promoted_team]
        instance = []
        for k in range (0, len(home_stats)):
            if (away_stats[k] == 0):
                ratio = (home_stats[k] + 1) / (away_stats[k] + 1)
            else:
                
                ratio = home_stats[k] / away_stats[k]
                
#             ratio = home_stats[k] - away_stats[k]
#             instance.append(ratio)
            instance.append(away_stats[k])
#             instance.append(home_stats[k] - away_stats[k])
            instance.append(home_stats[k])           
        outcome = results[(home_team, away_team)]
        if (outcome !=0):
            stats.append(instance)
            all_stats.append(instance)
            outcomes.append(outcome)
            all_outcomes.append(outcome)
    stats_pairs_by_season.append(stats)
    outcomes_by_season.append(outcomes)
    print(wins, losses, draws, draws/(wins+losses+draws))
print(len(all_stats))
print(len(all_outcomes))
print(all_outcomes)

print(all_stats)

STARTING OVER
173 108 99 0.26052631578947366
STARTING OVER
187 109 84 0.22105263157894736
577
577
[1, -1, -1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, -1, -1, 1, -1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, -1, -1,

In [25]:
num_classes = 2
x_train, x_test, y_train, y_test = train_test_split(all_stats, all_outcomes, test_size = 0.3)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

num_records = len(x_train)
num_features = len(x_train[0])


print("Training Feature matrix being passed into neural network has " + str(num_records) + 
      " vectors and " + str(num_features) + " parameters. ")

Training Feature matrix being passed into neural network has 322 vectors and 44 parameters. 


In [26]:
def fit_maxent_classifier(X, y, testFeatureVectors): 
    mod = LogisticRegression(fit_intercept=True, multi_class = 'multinomial', solver = 'newton-cg')
    mod.fit(X, y)
    pred = mod.predict(testFeatureVectors)
    # print(pred)
    return pred

In [27]:
pred = fit_maxent_classifier(x_train, y_train, x_val)


/anaconda3/envs/nlu/lib/python3.6/site-packages/sklearn/utils/optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [28]:
y_test_pred = fit_maxent_classifier(x_train, y_train, x_test)
print(y_test_pred)

correct = 0
total = 0

for i in range(0, len(y_test_pred)):
    if y_test_pred[i] == y_test[i]:
        correct+=1
    total+=1
    
print(correct)
print(total)
print(1.0 * correct / total)


[-1 -1 -1 -1  1 -1  1 -1  1  1  1  1  1 -1  1 -1 -1  1  1  1 -1 -1  1 -1
  1  1 -1  1  1  1  1  1  1 -1  1  1 -1  1  1 -1 -1  1 -1  1  1  1  1  1
  1  1 -1  1 -1 -1  1 -1 -1  1  1 -1 -1 -1  1  1 -1 -1 -1  1  1  1 -1  1
 -1 -1  1 -1  1 -1  1  1  1 -1  1  1  1  1  1  1  1  1 -1  1 -1  1  1 -1
  1  1 -1  1 -1  1  1  1  1  1 -1  1  1 -1 -1  1 -1 -1  1  1  1  1  1  1
  1  1  1  1  1  1 -1 -1  1 -1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 -1  1  1 -1  1 -1  1 -1  1  1 -1  1  1  1  1  1  1 -1 -1  1  1  1  1 -1
 -1  1  1 -1  1  1]
130
174
0.7471264367816092


/anaconda3/envs/nlu/lib/python3.6/site-packages/scipy/optimize/linesearch.py:312: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/anaconda3/envs/nlu/lib/python3.6/site-packages/sklearn/utils/optimize.py:195: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
